# 🎯 PID Auto-Tuning with Multi-Objective Optimization

> **Автоматическая настройка PID-регулятора с композитным критерием качества для авиационных систем управления**

Этот пример демонстрирует продвинутую оптимизацию коэффициентов PID-регулятора для продольной динамики самолёта Boeing 747. Вместо оптимизации по единственной метрике, используется **композитный критерий**, учитывающий множественные показатели качества переходного процесса.

## ✨ Ключевые особенности

- 🎯 **Мультиметричная оптимизация** — учёт 10+ показателей качества системы управления
- 🚀 **Быстрая сходимость** — использование Optuna TPE-алгоритма вместо Ray Tune
- ⚖️ **Гибкие ограничения** — настраиваемые жёсткие лимиты на перерегулирование, ошибку, колебания
- 📊 **Детальная аналитика** — полный отчёт по всем метрикам для лучших параметров
- 🔧 **Легкая настройка** — простое изменение весов и ограничений под конкретную задачу

## 📊 Оптимизируемые метрики

| Категория | Метрики | Описание |
|-----------|---------|----------|
| **Временные** | `settling_time_sec`, `rise_time_idx`, `peak_time_idx` | Скорость переходного процесса |
| **Точность** | `static_error`, `overshoot`, `max_dev` | Качество отслеживания |
| **Интегральные** | `ISE`, `IAE`, `ITAE` | Накопленная ошибка |
| **Устойчивость** | `oscillations`, `damping_degree` | Колебательность системы |
| **Композитный** | `performance_index` | Встроенный индекс качества |

## 🎯 Целевая функция

### Жёсткие ограничения (по умолчанию):
```python
MAX_OVERSHOOT_PERCENT = 10.0    # перерегулирование ≤ 10%
MAX_STATIC_ERROR_ABS = 0.5      # |статическая ошибка| ≤ 0.5°
MAX_OSCILLATIONS = 6            # количество колебаний ≤ 6
```

### Композитный скор (минимизируется):
```python
score = W_SETTLING × settling_time_sec +
        W_ITAE × ITAE +
        W_ISE × ISE +
        W_MAX_DEV × max_deviation +
        W_RISE × rise_time +
        W_DAMPING × damping_degree  # отрицательный вес
```

## 🚀 Быстрый старт

### 1. Запуск базового примера
```python
# Выполните ячейки по порядку:
# 1. Импорты и инициализация
# 2. Функция симуляции simulate_pid_step()
# 3. Настройка Optuna objective
# 4. Запуск оптимизации study.optimize()
# 5. Анализ результатов
```

### 2. Результат оптимизации
```python
# Пример вывода:
{
  'params': {'kp': 7.21, 'ki': 17.14, 'kd': 5.03},
  'composite_score': 15.42,
  'overshoot': 8.3,           # %
  'settling_time_sec': 12.5,  # сек
  'static_error': 0.12,       # град
  'ise': 245.8,
  'oscillations': 2
}
```

## ⚙️ Настройка под задачу

### Изменение ограничений:
```python
MAX_OVERSHOOT_PERCENT = 5.0     # более жёсткое ограничение
MAX_STATIC_ERROR_ABS = 1.0      # более мягкое ограничение
```

### Настройка весов композитного критерия:
```python
W_SETTLING = 0.4    # больший акцент на скорость
W_ITAE = 0.3        # меньший акцент на интегральную ошибку
W_DAMPING = -0.1    # сильнее поощрять затухание
```

### Параметры оптимизации:
```python
N_TRIALS = 500      # больше итераций для лучшего результата
SIM_TN = 30.0       # увеличить время моделирования
```

## 📈 Мониторинг процесса

Optuna предоставляет встроенный прогресс-бар и логирование:
```
Best trial: 42. Best value: 12.34: 67%|██████▋   | 134/200
[I 2025-08-31 18:33:25] Trial 134 finished with value: 12.34
```

## 🔬 Технические детали

- **Модель**: `LinearLongitudinalB747-v0` (продольная динамика Boeing 747)
- **Переменная состояния**: `theta` (угол тангажа)
- **Входной сигнал**: ступенчатое воздействие 5° на 10-м шаге
- **Алгоритм оптимизации**: TPE (Tree-structured Parzen Estimator)
- **Время моделирования**: 20 сек с шагом 0.01 сек

## 📋 Требования

```python
tensoraerospace >= 0.3.0
optuna >= 3.0.4
numpy >= 1.19.5
gymnasium >= 0.20.0
```

## 🤝 Вклад в развитие

Этот пример можно расширить:
- Добавить другие типы входных сигналов (синус, рампа)
- Реализовать робастную оптимизацию с учётом неопределённостей
- Интегрировать методы машинного обучения для предсказания качества
- Добавить визуализацию переходных процессов

## 📄 Лицензия

Этот пример распространяется под лицензией MIT. См. файл LICENSE для деталей.


## 📦 Импорт зависимостей

Подключаем все необходимые библиотеки:
- `PID` — класс ПИД-регулятора из TensorAeroSpace
- `numpy` — для численных вычислений
- `gymnasium` — интерфейс для сред моделирования
- `tensoraerospace.benchmark.function` — метрики качества переходных процессов
- `tensoraerospace.utils` и `tensoraerospace.signals` — утилиты для генерации временных рядов и сигналов


In [12]:
from tensoraerospace.agent.pid import PID
import numpy as np
from ray import train, tune

from tensoraerospace.envs.f16.linear_longitudial import LinearLongitudinalF16
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

import gymnasium as gym 
from tensoraerospace.benchmark.function import overshoot, settling_time, static_error

## ⚙️ Базовые параметры моделирования

Настраиваем основные параметры для симуляции:
- `dt = 0.01` — шаг дискретизации (10 мс)
- `tn = 20` — общее время моделирования (20 секунд)
- `tp` — массив временных точек
- `reference_signals` — эталонный сигнал (ступенька 5° на 10-м шаге)


In [13]:
dt = 0.01  # Дискретизация
tp = generate_time_period(tn=20, dt=dt) # Временной периуд
tps = convert_tp_to_sec_tp(tp, dt=dt)
number_time_steps = len(tp) # Количество временных шагов
reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1]) # Заданный сигнал

## 🔬 Функция симуляции PID с полным набором метрик

Это ключевая функция, которая:
1. **Создаёт среду моделирования** Boeing 747 с заданными параметрами
2. **Инициализирует PID-регулятор** с переданными коэффициентами
3. **Запускает симуляцию** переходного процесса
4. **Вычисляет все метрики качества**:
   - Временные характеристики (время установления, нарастания, пик)
   - Показатели точности (перерегулирование, статическая ошибка)
   - Интегральные критерии (ISE, IAE, ITAE)
   - Характеристики устойчивости (колебания, затухание)

Эта функция используется оптимизатором для оценки качества каждого набора PID-коэффициентов.


In [14]:
# Вспомогательная функция симуляции шага с вычислением метрик
import warnings
warnings.filterwarnings("ignore")

from tensoraerospace.benchmark.function import (
    overshoot as metric_overshoot,
    settling_time as metric_settling_time,
    damping_degree as metric_damping_degree,
    rise_time as metric_rise_time,
    peak_time as metric_peak_time,
    maximum_deviation as metric_max_dev,
    integral_absolute_error as metric_iae,
    integral_squared_error as metric_ise,
    integral_time_absolute_error as metric_itae,
    oscillation_count as metric_osc_count,
    performance_index as metric_perf_idx,
)


def simulate_pid_step(kp: float, ki: float, kd: float,
                      dt: float = 0.01,
                      tn: float = 20.0,
                      step_deg: float = 5.0,
                      step_time_idx: int = 10,
                      env_id: str = 'LinearLongitudinalF16-v0',
                      track: str = 'alpha',
                      settle_thr: float = 0.05) -> dict:
    """
    Запускает шаговую реакцию для PID и возвращает метрики переходного процесса.

    Returns dict:
      - overshoot: процент, >0 означает перерегулирование
      - settling_time_idx: индекс времени установления (по шагам)
      - settling_time_sec: время установления в секундах
      - static_error: статическая ошибка (в градусах)
      - ise, iae, itae: интегральные критерии
      - rise_time_idx, peak_time_idx: индексы характеристик
      - max_dev: максимальное отклонение
      - oscillations: число колебаний
      - damping_degree: степень затухания
      - performance_index: составной индекс качества
    """
    hist = []
    tp = generate_time_period(tn=tn, dt=dt)
    tps = convert_tp_to_sec_tp(tp, dt=dt)
    number_time_steps = len(tp)

    reference_signals = np.reshape(
        unit_step(degree=step_deg, tp=tp, time_step=step_time_idx, output_rad=True),
        [1, -1]
    )

    env = gym.make(
        env_id,
        number_time_steps=number_time_steps,
        initial_state=[ [0], [0], [0]],
        reference_signal=reference_signals,
    state_space=["theta", "alpha", "q"],
    output_space=["theta", "alpha", "q"],
        tracking_states=[track],
        use_reward=False,
    )
    env.reset()

    pid = PID(kp=kp, ki=ki, kd=kd, dt=dt)
    xt = np.array([[np.deg2rad(0)], [0]])

    for step in range(number_time_steps - 2):
        setpoint = reference_signals[0, step]
        hist.append(xt[0, 0])
        ut = pid.select_action(setpoint, xt[0, 0])
        xt, reward, terminated, truncated, info = env.step(np.array([ut.item()]))

    system_signal_deg = env.unwrapped.model.get_state(track, to_deg=True)
    control_signal_deg = np.rad2deg(reference_signals[0])

    # Усечение до длины моделирования (на всякий случай)
    N = min(len(system_signal_deg), len(control_signal_deg))
    system_signal_deg = system_signal_deg[:N]
    control_signal_deg = control_signal_deg[:N]

    os_percent = float(abs(metric_overshoot(control_signal_deg, system_signal_deg)))
    st_idx = int(metric_settling_time(control_signal_deg, system_signal_deg, threshold=settle_thr))
    st_sec = st_idx * dt

    # статическая ошибка (последние 10%)
    r_final = float(np.mean(control_signal_deg[int(0.9 * N):]))
    y_final = float(np.mean(system_signal_deg[int(0.9 * N):]))
    static_err = r_final - y_final

    # Дополнительные метрики
    ise_val = float(metric_ise(control_signal_deg, system_signal_deg))
    iae_val = float(metric_iae(control_signal_deg, system_signal_deg))
    itae_val = float(metric_itae(control_signal_deg, system_signal_deg, dt=dt))
    rise_idx = metric_rise_time(control_signal_deg, system_signal_deg) or N
    peak_idx = metric_peak_time(system_signal_deg) or np.argmax(system_signal_deg)
    max_dev_val = float(metric_max_dev(control_signal_deg, system_signal_deg))
    osc_count = int(metric_osc_count(system_signal_deg))
    damping = float(metric_damping_degree(system_signal_deg))
    perf_idx = float(metric_perf_idx(control_signal_deg, system_signal_deg, dt=dt))

    return {
        "overshoot": os_percent,
        "settling_time_idx": st_idx,
        "settling_time_sec": st_sec,
        "static_error": static_err,
        "ise": ise_val,
        "iae": iae_val,
        "itae": itae_val,
        "rise_time_idx": int(rise_idx),
        "peak_time_idx": int(peak_idx),
        "max_dev": max_dev_val,
        "oscillations": osc_count,
        "damping_degree": damping,
        "performance_index": perf_idx,
    }


## 🎯 Настройка оптимизации с композитным критерием

Здесь определяются:

### Жёсткие ограничения качества:
- Максимальное перерегулирование, статическая ошибка, количество колебаний

### Веса композитного критерия:
- `W_SETTLING` — важность времени установления
- `W_ITAE` — важность интегральной ошибки по времени
- `W_ISE` — важность квадратичной ошибки
- `W_DAMPING` — поощрение затухания (отрицательный вес)

### Функции:
- `composite_score()` — вычисляет взвешенную сумму метрик
- `objective()` — целевая функция для Optuna, применяет ограничения и возвращает композитный скор
- `study` — объект исследования Optuna для минимизации


In [15]:
# Оптимизация с Optuna: составной критерий качества с ограничениями
import optuna

# Ограничения (жесткие)
MAX_OVERSHOOT_PERCENT = 10.0  # %
MAX_STATIC_ERROR_ABS = 0.5    # градусы
MAX_OSCILLATIONS = 6          # количество колебаний
SETTLING_THRESHOLD = 0.05     # коридор 5%

# Веса составного индекса (настраиваемые)
W_SETTLING = 0.35
W_ITAE = 0.25
W_ISE = 0.15
W_MAX_DEV = 0.1
W_RISE = 0.1
W_DAMPING = -0.05  # поощряем большее затухание (минусовой вес)

SIM_DT = 0.01
SIM_TN = 20.0


def composite_score(m: dict) -> float:
    # Нормализации (простые масштабы, можно улучшить под домен)
    settling_s = m["settling_time_sec"]
    itae = m["itae"]
    ise = m["ise"]
    max_dev = abs(m["max_dev"])  # градусы
    rise = m["rise_time_idx"] * SIM_DT
    damping = max(0.0, m["damping_degree"])  # >=0

    score = (
        W_SETTLING * settling_s +
        W_ITAE * itae +
        W_ISE * ise +
        W_MAX_DEV * max_dev +
        W_RISE * rise +
        W_DAMPING * damping
    )
    return float(score)


def objective(trial: optuna.Trial) -> float:
    kp = trial.suggest_float("kp", -20.0, 20.0)
    ki = trial.suggest_float("ki", -20.0, 20.0)
    kd = trial.suggest_float("kd", -20.0, 20.0)

    m = simulate_pid_step(kp=kp, ki=ki, kd=kd,
                          dt=SIM_DT, tn=SIM_TN,
                          step_deg=5.0, step_time_idx=10,
                          settle_thr=SETTLING_THRESHOLD)

    # Жесткие ограничения качества
    if m["overshoot"] > MAX_OVERSHOOT_PERCENT:
        return 1e6 + m["overshoot"]
    if abs(m["static_error"]) > MAX_STATIC_ERROR_ABS:
        return 1e6 + abs(m["static_error"]) * 1e3
    if m["oscillations"] > MAX_OSCILLATIONS:
        return 1e6 + m["oscillations"] * 1e3

    # Составной индекс (чем меньше, тем лучше)
    return composite_score(m)


study = optuna.create_study(direction="minimize")


[I 2025-08-31 19:57:38,442] A new study created in memory with name: no-name-5019d62e-e70d-47c6-9fde-6864ae3bada1


## 🚀 Запуск процесса оптимизации

Эта ячейка запускает поиск оптимальных PID-коэффициентов:

- **`N_TRIALS = 200`** — количество попыток оптимизации (можно увеличить для лучшего результата)
- **`show_progress_bar=True`** — показывает прогресс выполнения
- **TPE алгоритм** — Optuna использует Tree-structured Parzen Estimator для умного поиска

Во время выполнения вы увидите:
- Прогресс-бар с лучшим найденным значением
- Логи каждого завершённого испытания
- Текущие лучшие параметры

⏱️ **Время выполнения**: ~15-30 секунд для 200 испытаний


In [16]:
# Запуск оптимизации (регулируйте n_trials по времени)
from tabnanny import verbose


N_TRIALS = 5000  # при необходимости увеличить

study.optimize(objective, n_jobs=10, n_trials=N_TRIALS, show_progress_bar=False)


[I 2025-08-31 19:57:39,261] Trial 9 finished with value: 1000621.8332029759 and parameters: {'kp': -6.608693664162644, 'ki': 1.1240952520218386, 'kd': 12.808370870203674}. Best is trial 9 with value: 1000621.8332029759.
[I 2025-08-31 19:57:39,331] Trial 4 finished with value: 1000060.5443146185 and parameters: {'kp': -6.463134343743789, 'ki': -1.7353959226892002, 'kd': -14.912191967770099}. Best is trial 4 with value: 1000060.5443146185.
[I 2025-08-31 19:57:39,451] Trial 6 finished with value: 1000099.9870615574 and parameters: {'kp': 19.331025456702953, 'ki': -2.25749662072743, 'kd': -18.57433999114086}. Best is trial 4 with value: 1000060.5443146185.
[I 2025-08-31 19:57:39,463] Trial 2 finished with value: 1000099.9870615574 and parameters: {'kp': 5.408315464821495, 'ki': 3.2752136903499007, 'kd': -10.903889940030096}. Best is trial 4 with value: 1000060.5443146185.
[I 2025-08-31 19:57:39,522] Trial 5 finished with value: 1002055.2010576456 and parameters: {'kp': -19.098786638182336,

KeyboardInterrupt: 

## 📊 Извлечение лучших результатов

Получаем оптимальные параметры PID-регулятора:

- **`best_params`** — словарь с найденными коэффициентами `{'kp': ..., 'ki': ..., 'kd': ...}`
- **`best_value`** — значение композитного критерия для лучшего решения

Эти параметры представляют лучший компромисс между всеми метриками качества с учётом заданных ограничений.


In [ ]:
# Вывод лучших коэффициентов PID и целевого значения
best = study.best_trial
best_params = best.params
best_value = best.value
best_params, best_value


({'kp': 2.8845310033055416,
  'ki': -0.5082403780315914,
  'kd': -11.326428032256596},
 21714.53067855468)

## 🔍 Валидация и детальный анализ результатов

Финальная проверка найденных параметров:

1. **Повторная симуляция** с лучшими коэффициентами для подтверждения результата
2. **Расчёт композитного скора** для проверки соответствия оптимизации
3. **Полный отчёт** со всеми метриками качества:
   - Параметры PID
   - Композитный скор
   - Все 10+ метрик переходного процесса

Этот отчёт позволяет убедиться, что система управления действительно соответствует всем требованиям качества, а не только одной метрике.


In [ ]:
# Валидация: запуск симуляции с лучшими параметрами и отчёт метрик
val_metrics = simulate_pid_step(kp=best_params['kp'], ki=best_params['ki'], kd=best_params['kd'],
                                dt=SIM_DT, tn=SIM_TN,
                                step_deg=5.0, step_time_idx=10,
                                settle_thr=SETTLING_THRESHOLD)

# Рассчёт составного индекса для лучших параметров
val_score = composite_score(val_metrics)
{"params": best_params, "composite_score": val_score, **val_metrics}


{'params': {'kp': -14.290139135229715,
  'ki': -8.240470780203491,
  'kd': -1.2991634935096958},
 'composite_score': 30638.45313693336,
 'overshoot': 5.951972760698236,
 'settling_time_idx': 1827,
 'settling_time_sec': 18.27,
 'static_error': 0.29400188189360676,
 'ise': 58439.510876291504,
 'iae': 8585.828617641706,
 'itae': 87459.72548975996,
 'rise_time_idx': 204,
 'peak_time_idx': 300,
 'max_dev': 9.966330496445853,
 'oscillations': 2,
 'damping_degree': -0.07243534922969574,
 'performance_index': 58360.88494097273}

---

## 📚 Дополнительно: Оригинальная функция оптимизации (для сравнения)

Ниже представлена оригинальная функция `env_optimization()` из первоначальной версии кода. Она использует:
- Простой составной критерий из статической ошибки, перерегулирования и времени установления
- Жёсткое ограничение на максимальное значение сигнала (6 радиан)
- Другие веса и нормализацию

Эта функция оставлена для сравнения подходов и демонстрации эволюции алгоритма оптимизации.


## 🧪 Тестирование оригинальной функции

Простой тест оригинальной функции оптимизации с базовыми коэффициентами PID (kp=1, ki=1, kd=1):

Этот вызов показывает, как работала старая версия алгоритма и какие значения она возвращала для тех же входных параметров.


## 📊 Оригинальная целевая функция Ray Tune (устаревшая)

Это оригинальная целевая функция для Ray Tune из первой версии кода:
- Принимает конфигурацию в формате Ray Tune
- Использует `train.report()` для отчётности
- Вызывает старую функцию `env_optimization()`

**Примечание**: Эта функция больше не используется в основном алгоритме, но оставлена для справки.


## ⚙️ Настройка Ray Tune (устаревшая)

Оригинальная настройка оптимизатора Ray Tune:
- Использует `tune.Tuner()` с большим количеством испытаний (40000)
- Настроен на минимизацию `mean_loss`
- Диапазоны параметров от -20 до 20 для всех коэффициентов PID

**Примечание**: Эта ячейка больше не выполняется, так как мы перешли на Optuna для лучшей производительности и стабильности.
